In [1]:
input_file = "dream.txt"

import json
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import tensorflow_probability as tfp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

# TODO: Change root path
root = "/Users/jay/Desktop/Bachelorarbeit"

import sys

sys.path.append(f"{root}/Implementation")
from dependencies.hbv_sask.model import HBVSASKModel as hbvmodel
from src.execute_model import run_model_single_parameter_node
from src.construct_model import get_model


ndims = 7
dims = ["TT", "C0", "beta", "ETF", "FC", "FRAC", "K2"]
colors = sns.color_palette(n_colors=ndims)

testConfigPath = f"{root}/test_config.json"
with open(testConfigPath, "r") as file:
    run_config = json.load(file)

configPath = run_config["configPath"]
basis = run_config["basis"]
model = get_model(configPath, basis)

start_date: 2004-01-01 00:00:00
start_date_predictions: 2005-01-01 00:00:00
end_date: 2006-01-01 00:00:00
simulation length: 365
full_data_range is 732 hours including spin_up_length of 366 hours
simulation_range is of length 366 hours


In [2]:
# Construct params
configurationObject = model.configurationObject
param_lower = []
param_upper = []
for param in configurationObject["parameters"]:
    if param["distribution"] == "Uniform":
        param_lower.append(param["lower"])
        param_upper.append(param["upper"])
    else:
        raise NotImplementedError(
            f"Sorry, the distribution {param['distribution']} is not supported yet"
        )
param_lower = np.array(param_lower)
param_upper = np.array(param_upper)

In [3]:
samples = pd.read_csv(f"{input_file}")
samples

,TT,C0,beta,ETF,FC,FRAC,K2
0,2.768674,2.474903,1.857740,0.056647,233.712875,0.205748,0.044582
1,2.768674,2.474903,1.857740,0.056647,233.712875,0.205748,0.044582
2,2.768674,2.474903,1.857740,0.056647,233.712875,0.205748,0.044582
3,2.768674,2.474903,1.857740,0.056647,233.712875,0.205748,0.044582
4,2.768674,2.474903,1.857740,0.056647,233.712875,0.205748,0.044582
...,...,...,...,...,...,...,...
2667,2.633615,2.224877,1.850932,0.020617,231.610444,0.219667,0.049862
2668,2.633615,2.224877,1.850932,0.020617,231.610444,0.219667,0.049862
2669,2.633615,2.224877,1.850932,0.020617,231.610444,0.219667,0.049862
2670,2.633615,2.224877,1.850932,0.020617,231.610444,0.219667,0.049862


# Plotting

In [4]:
fig = make_subplots(rows=2, cols=4)

# Histogram and KDE
for i, col in enumerate(samples.columns):
    row = (i // 4) + 1
    col_idx = (i % 4) + 1

    # Histogram
    fig.add_trace(
        go.Histogram(x=samples[col], name=col, histnorm="probability density"),
        row=row,
        col=col_idx,
    )

    # KDE calculation
    kde = gaussian_kde(samples[col])
    x_values = np.linspace(samples[col].min(), samples[col].max(), 300)
    kde_values = kde(x_values)
    fig.add_trace(
        go.Scatter(
            x=x_values, y=kde_values, mode="lines", name=f"KDE {col}", showlegend=False
        ),
        row=row,
        col=col_idx,
    )

# Update layout
fig.update_layout(
    height=500,
    width=1200,
    title_text="DREAM: Parameters Overview",
)
fig.show()

In [5]:
fig = make_subplots(rows=2, cols=4)

for i, col in enumerate(samples.columns):
    row = (i // 4) + 1
    col_idx = (i % 4) + 1

    fig.add_trace(
        go.Box(
            y=samples[col],
            name=col,
            boxpoints="all",
            jitter=0.5,
            whiskerwidth=0.2,
            marker_size=2,
            line_width=1,
        ),
        row=row,
        col=col_idx,
    )

# Update layout
fig.update_layout(
    height=500,
    width=1200,
    title_text="DREAM: Boxplots of Parameters",
)
fig.show()

# Sampling

In [6]:
# Sampling Max
param_vec = []
for i in range(len(samples.loc[0])):
    values, counts = np.unique(samples.iloc[:, i], return_counts=True)
    ind = np.argmax(counts)
    param_vec.append(values[ind])

_, posterior_max, _, _ = run_model_single_parameter_node(model, param_vec)

[HVBSASK INFO] [0] parameters: [[2.614559771915773, 2.2104799537019355, 1.781313089735102, 0.002510710392677947, 232.52224236231393, 0.21608871315432185, 0.04986176814543106]]


In [7]:
# Mean sampling from posterior
sample_param = []
for i in range(7):
    sample_param.append(np.random.choice(samples.iloc[:, i], 1000))
sample_param = np.array(sample_param).T

posterior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    posterior.append(y_model)

print(np.array(posterior).shape)
posterior_mean = np.mean(np.array(posterior), axis=0)

[HVBSASK INFO] [0] parameters: [array([2.75855251e+00, 2.38929400e+00, 2.10186254e+00, 2.60702481e-02,
       2.34156124e+02, 2.21314212e-01, 4.89923366e-02])]
[HVBSASK INFO] [0] parameters: [array([2.62684180e+00, 2.42805389e+00, 1.89277987e+00, 2.31246421e-02,
       2.34091519e+02, 2.16880883e-01, 4.98039208e-02])]
[HVBSASK INFO] [0] parameters: [array([2.76867449e+00, 2.44721743e+00, 1.74193253e+00, 1.29904671e-02,
       2.33138825e+02, 2.20337707e-01, 4.96949281e-02])]
[HVBSASK INFO] [0] parameters: [array([2.99514267e+00, 2.33115241e+00, 1.78903367e+00, 2.14511246e-02,
       2.37544975e+02, 2.27322982e-01, 4.91105836e-02])]
[HVBSASK INFO] [0] parameters: [array([2.61817289e+00, 2.28551926e+00, 1.95948217e+00, 2.19226602e-02,
       2.31804896e+02, 2.24206092e-01, 4.88070980e-02])]
[HVBSASK INFO] [0] parameters: [array([2.34136446e+00, 2.36630934e+00, 1.82004562e+00, 1.17090029e-02,
       2.45305837e+02, 2.14293783e-01, 4.91581015e-02])]
[HVBSASK INFO] [0] parameters: [array([2

In [8]:
# Mean sampling from prior
sample_param = []
for i in range(7):
    uni = tfp.distributions.Uniform(low=param_lower[i], high=param_upper[i]).sample(1)
    sample_param.append(uni)
sample_param = np.array(sample_param).T

prior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    prior.append(y_model)

_, _, measured_data, _ = run_model_single_parameter_node(model, np.array(vec))

prior_means = np.mean(np.array(prior), axis=0)

[HVBSASK INFO] [0] parameters: [array([2.85954558e+00, 1.45252261e-01, 2.22954188e+00, 3.10888873e-01,
       1.11549348e+02, 8.03260670e-01, 3.11138257e-02])]
[HVBSASK INFO] [0] parameters: [array([2.85954558e+00, 1.45252261e-01, 2.22954188e+00, 3.10888873e-01,
       1.11549348e+02, 8.03260670e-01, 3.11138257e-02])]


2024-08-07 12:20:31.966554: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-08-07 12:20:31.966584: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-08-07 12:20:31.966588: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-08-07 12:20:31.966610: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-07 12:20:31.966621: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
fig = go.Figure()
model.get_start_date
dates = model.get_simulation_range()
fig.add_trace(
    go.Scatter(
        x=dates,
        y=prior_means,
        mode="lines",
        name="Prior Mean",
        line=dict(color="lightgrey"),
    )
)
fig.add_trace(
    go.Scatter(x=dates, y=posterior_mean, mode="lines", name="Posterior Mean")
)
fig.add_trace(go.Scatter(x=dates, y=posterior_max, mode="lines", name="Posterior Max"))
fig.add_trace(go.Scatter(x=dates, y=measured_data, mode="lines", name="Measured Data"))

# Update layout
fig.update_layout(
    title="DREAM: Bayesian Inference Result Comparison",
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Time Series",
    hovermode="x unified",
    template="plotly_white",
)

fig.show()

In [10]:
def rmse(result, target):
    diff = result - target
    aggr = 0
    for i in range(len(diff)):
        aggr += diff[i] ** 2
    rmse = (aggr / (len(diff))) ** 0.5
    return rmse


def mae(result, target):
    return np.absolute(result - target).mean()


print(f"RMSE of Posterior Mean: {rmse(posterior_mean, measured_data)}")
print(f"RMSE of Posterior Max: {rmse(posterior_max, measured_data)}")
print(f"MAE of Posterior Mean: {mae(posterior_mean, measured_data)}")
print(f"MAE of Posterior Max: {mae(posterior_max, measured_data)}")

RMSE of Posterior Mean: 20.472362170131376
RMSE of Posterior Max: 20.48601441220307
MAE of Posterior Mean: 9.354213645852623
MAE of Posterior Max: 9.231561092568036
